In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import xgboost
import pickle as pkl
from sklearn.model_selection import cross_val_score, GridSearchCV
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings ("ignore")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.12/site-

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: initialization failed

In [2]:
data = pd.read_csv('newTrain.csv')
data

,S.No.,datetime,temperature,humidity,tempDegree,humidityDegree,fanSpeed
0,1,1/4/21 0:01,32,34,1,0,2
1,2,1/4/21 0:01,32,35,1,0,2
2,3,1/4/21 0:01,32,35,1,0,2
3,4,1/4/21 0:02,32,35,1,0,2
4,5,1/4/21 0:04,32,34,1,0,2
...,...,...,...,...,...,...,...
79371,79380,31/07/2021 9:54,32,92,1,2,4
79372,79381,31/07/2021 9:55,31,93,1,2,4
79373,79382,31/07/2021 9:55,32,92,1,2,4
79374,79383,31/07/2021 9:56,31,93,1,2,4


In [3]:
print('Dataset shape:', data.shape)

Dataset shape: (79376, 7)


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79376 entries, 0 to 79375
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   S.No.           79376 non-null  int64 
 1   datetime        79376 non-null  object
 2   temperature     79376 non-null  int64 
 3   humidity        79376 non-null  int64 
 4   tempDegree      79376 non-null  int64 
 5   humidityDegree  79376 non-null  int64 
 6   fanSpeed        79376 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 4.2+ MB


## Handling Missing Values

In [6]:
data.isnull().sum()

S.No.             0
datetime          0
temperature       0
humidity          0
tempDegree        0
humidityDegree    0
fanSpeed          0
dtype: int64

In [7]:
data.isnull().any()

S.No.             False
datetime          False
temperature       False
humidity          False
tempDegree        False
humidityDegree    False
fanSpeed          False
dtype: bool

In [8]:
data["tempDegree"].unique()

array([1, 2, 0])

In [9]:
data["humidityDegree"].unique()

array([0, 1, 2])

In [10]:
data["fanSpeed"].unique()

array([2, 3, 5, 4, 1, 6])

### There are no null values

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79376 entries, 0 to 79375
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   S.No.           79376 non-null  int64 
 1   datetime        79376 non-null  object
 2   temperature     79376 non-null  int64 
 3   humidity        79376 non-null  int64 
 4   tempDegree      79376 non-null  int64 
 5   humidityDegree  79376 non-null  int64 
 6   fanSpeed        79376 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 4.2+ MB


In [13]:
data["fanSpeed"].value_counts()

fanSpeed
3    26156
5    18402
4    18005
2     8753
6     8045
1       15
Name: count, dtype: int64

In [14]:
data = data.drop(['S.No.', 'datetime', 'tempDegree', 'humidityDegree'], axis=1)

In [15]:
data

,temperature,humidity,fanSpeed
0,32,34,2
1,32,35,2
2,32,35,2
3,32,35,2
4,32,34,2
...,...,...,...
79371,32,92,4
79372,31,93,4
79373,32,92,4
79374,31,93,4


### Exploratory Data Analysis

## Checking Duplicated Values

In [18]:
data.duplicated().sum()

78528

In [19]:
x = data.drop('fanSpeed', axis=1)
x

,temperature,humidity
0,32,34
1,32,35
2,32,35
3,32,35
4,32,34
...,...,...
79371,32,92
79372,31,93
79373,32,92
79374,31,93


In [20]:
y = data['fanSpeed']
y

0        2
1        2
2        2
3        2
4        2
        ..
79371    4
79372    4
79373    4
79374    4
79375    4
Name: fanSpeed, Length: 79376, dtype: int64

In [21]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3, random_state=1234)

In [22]:
x_train

,temperature,humidity
63352,32,37
8652,35,56
34581,34,57
56738,34,82
78453,32,58
...,...,...
55985,35,75
32399,32,62
60620,33,80
34086,33,66


In [23]:
y_train

63352    2
8652     5
34581    5
56738    6
78453    3
        ..
55985    6
32399    3
60620    4
34086    3
58067    6
Name: fanSpeed, Length: 55563, dtype: int64

In [24]:

def train_models_eval(x_train,y_train,fts):
    print("\n---›RANDOM FOREST" )
    rf = RandomForestClassifier(random_state=1234)
    rf.fit(x_train[fts], y_train)
    y_pred=rf.predict(x_test[fts])
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    print("SCORE:",rf.score(x_test[fts],y_test))
    
    print("\n---›DECISION TREE")
    dtf = DecisionTreeClassifier(random_state=1234)
    dtf.fit(x_train[fts], y_train)
    y_pred=dtf.predict(x_test[fts])
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    print("SCORE:",dtf.score(x_test[fts],y_test))
    
    print(" \n--->KNN")
    knn = KNeighborsClassifier()
    knn.fit(x_train[fts], y_train)
    y_pred=knn.predict(x_test[fts])
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    print("SCORE:",knn.score(x_test[fts],y_test))
    
    print("\n---›EXTRAS TREES CLASSIFIER")
    etc = ExtraTreesClassifier(random_state=1234)
    etc.fit(x_train[fts],y_train)
    y_pred=etc.predict(x_test[fts])
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    print("SCORE:",etc.score(x_test[fts],y_test))
    
    #print("\n---›XGBOOST" )
    #xgb = xgboost.XGBClassifier ()
    #xgb.fit(x_train[fts], y_train)
    #y_pred=xgb.predict(x_test[fts])
    #print(confusion_matrix(y_test,y_pred))
    #print(classification_report(y_test,y_pred))
    #print("SCORE:", xgb.score(x_test[fts],y_test))
    
    return rf,dtf,knn,etc

In [25]:
fts = data.drop('fanSpeed', axis=1).columns
rf,dtf,knn,etc= train_models_eval(x_train,y_train,fts)


---›RANDOM FOREST
[[   4    0    0    0    0    0]
 [   0 2642    0    0    0    0]
 [   0    0 7848    0    0    0]
 [   0    0    0 5429    0    0]
 [   0    0    0    0 5468    0]
 [   0    0    0    0    0 2422]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00      2642
           3       1.00      1.00      1.00      7848
           4       1.00      1.00      1.00      5429
           5       1.00      1.00      1.00      5468
           6       1.00      1.00      1.00      2422

    accuracy                           1.00     23813
   macro avg       1.00      1.00      1.00     23813
weighted avg       1.00      1.00      1.00     23813

SCORE: 1.0

---›DECISION TREE
[[   4    0    0    0    0    0]
 [   0 2642    0    0    0    0]
 [   0    0 7848    0    0    0]
 [   0    0    0 5429    0    0]
 [   0    0    0    0 5468    0]
 [   0    0    0    0    0 2422]]
          

### SMOTE

In [27]:
print(y_train.value_counts())

fanSpeed
3    18308
5    12934
4    12576
2     6111
6     5623
1       11
Name: count, dtype: int64


In [28]:
smote = SMOTE()

In [29]:
x_res_train,y_res_train = smote.fit_resample(x_train,y_train)

In [30]:
x_res_train

,temperature,humidity
0,32,37
1,35,56
2,34,57
3,34,82
4,32,58
...,...,...
109843,34,82
109844,35,77
109845,34,90
109846,35,73


In [31]:
y_res_train

0         2
1         5
2         5
3         6
4         3
         ..
109843    6
109844    6
109845    6
109846    6
109847    6
Name: fanSpeed, Length: 109848, dtype: int64

In [32]:
rf,dtf,knn,etc = train_models_eval(x_res_train,y_res_train,fts)


---›RANDOM FOREST
[[   4    0    0    0    0    0]
 [   0 2642    0    0    0    0]
 [   0    0 7848    0    0    0]
 [   0    0    0 5429    0    0]
 [   0    0    0    0 5468    0]
 [   0    0    0    0    0 2422]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00      2642
           3       1.00      1.00      1.00      7848
           4       1.00      1.00      1.00      5429
           5       1.00      1.00      1.00      5468
           6       1.00      1.00      1.00      2422

    accuracy                           1.00     23813
   macro avg       1.00      1.00      1.00     23813
weighted avg       1.00      1.00      1.00     23813

SCORE: 1.0

---›DECISION TREE
[[   4    0    0    0    0    0]
 [   0 2642    0    0    0    0]
 [   0    0 7848    0    0    0]
 [   0    0    0 5429    0    0]
 [   0    0    0    0 5468    0]
 [   0    0    0    0    0 2422]]
          

In [33]:
from collections import Counter
print(Counter(y_train))  # Check the class distribution after SMOTE


Counter({3: 18308, 5: 12934, 4: 12576, 2: 6111, 6: 5623, 1: 11})


In [34]:
from imblearn.over_sampling import BorderlineSMOTE

smote = BorderlineSMOTE()
X_train_balanced, y_train_balanced = smote.fit_resample(x_train, y_train)


In [35]:
# Define the features to be used for training
fts = x_train.columns  # Use all features or specify the feature names as needed

# Train and evaluate models
rf, dtf, knn, etc= train_models_eval(X_train_balanced, y_train_balanced, fts)



---›RANDOM FOREST
[[   4    0    0    0    0    0]
 [   0 2642    0    0    0    0]
 [   0    0 7848    0    0    0]
 [   0    0    0 5429    0    0]
 [   0    0    0    0 5468    0]
 [   0    0    0    0    0 2422]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00      2642
           3       1.00      1.00      1.00      7848
           4       1.00      1.00      1.00      5429
           5       1.00      1.00      1.00      5468
           6       1.00      1.00      1.00      2422

    accuracy                           1.00     23813
   macro avg       1.00      1.00      1.00     23813
weighted avg       1.00      1.00      1.00     23813

SCORE: 1.0

---›DECISION TREE
[[   4    0    0    0    0    0]
 [   0 2642    0    0    0    0]
 [   0    0 7848    0    0    0]
 [   0    0    0 5429    0    0]
 [   0    0    0    0 5468    0]
 [   0    0    0    0    0 2422]]
          

In [36]:
from sklearn.tree import DecisionTreeClassifier
import joblib

# Assuming dtf is already trained
# Save the model to a file named 'dtf.pkl'
joblib.dump(dtf, 'dtf.pkl')

# To load the model later
# dtf_loaded = joblib.load('dtf.pkl')


['dtf.pkl']

In [37]:
from sklearn.tree import export_text

tree_rules = export_text(dtf, feature_names=fts)
print(tree_rules)


|--- humidity <= 72.50
|   |--- temperature <= 33.50
|   |   |--- temperature <= 26.50
|   |   |   |--- humidity <= 45.50
|   |   |   |   |--- temperature <= 25.50
|   |   |   |   |   |--- humidity <= 44.50
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- humidity >  44.50
|   |   |   |   |   |   |--- temperature <= 24.50
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- temperature >  24.50
|   |   |   |   |   |   |   |--- class: 2
|   |   |   |   |--- temperature >  25.50
|   |   |   |   |   |--- class: 2
|   |   |   |--- humidity >  45.50
|   |   |   |   |--- temperature <= 25.50
|   |   |   |   |   |--- class: 2
|   |   |   |   |--- temperature >  25.50
|   |   |   |   |   |--- class: 3
|   |   |--- temperature >  26.50
|   |   |   |--- humidity <= 45.50
|   |   |   |   |--- class: 2
|   |   |   |--- humidity >  45.50
|   |   |   |   |--- class: 3
|   |--- temperature >  33.50
|   |   |--- class: 5
|--- humidity >  72.50
|   |--- temperature <= 33

In [1]:
import joblib
import numpy as np

# Load the saved model
dtf = joblib.load('dtf.pkl')

sample_input = np.array([[36,89]]) 

# Make a prediction
prediction = dtf.predict(sample_input)
print("Prediction:", prediction)

Prediction: [6]


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [5]:
pip install "numpy<2"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 723.7 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pickle

# Load the model
with open('dtf_updated.pkl', 'rb') as f:
    model = pickle.load(f)

# Test if the model has the predict attribute
try:
    sample_data = [[34, 45]]  # Replace with sample values for temperature and humidity
    prediction = model.predict(sample_data)
    print("Sample prediction:", prediction)
except AttributeError:
    print("The model does not have a predict method.")


Sample prediction: [5.]


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


In [3]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import pickle

# Load the dataset
data = pd.read_csv('newTrain.csv')

# Split features and target
X = data[['temperature', 'humidity']]
y = data['fanSpeed']

# Initialize and train the decision tree model
model = DecisionTreeRegressor()
model.fit(X, y)

# Save the trained model as dtf.pkl
with open('dtf_updated.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model saved as dtf.pkl")


Model saved as dtf.pkl


In [13]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
import pickle

# Load the dataset
data = pd.read_csv('newTrain.csv')

# Split features and target
X = data[['temperature', 'humidity']]
y = data['fanSpeed']

# Initialize and train the KNN model
knn_model = KNeighborsRegressor(n_neighbors=5)
knn_model.fit(X, y)

# Save the trained KNN model as knn.pkl
with open('knn.pkl', 'wb') as file:
    pickle.dump(knn_model, file)

print("KNN model saved as knn.pkl")


KNN model saved as knn.pkl


In [7]:
import joblib
import numpy as np
import pandas as pd

# Load the saved DecisionTreeRegressor model
dtf_model = joblib.load(r'dtf.pkl')

# Sample input for prediction (temperature, humidity)
sample_input = np.array([[36, 89]])

# Convert the input to a DataFrame with the same column names as the training data
input_df = pd.DataFrame(sample_input, columns=['temperature', 'humidity'])

# Predict with the model using the DataFrame input
prediction = dtf_model.predict(input_df)

# Print the prediction
print(prediction)


[6]


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
import sklearn
print(sklearn.__version__)


1.5.2
